# CMEMS WMS Data Retrieval
Construction and application of queries to retrieve data directly from Copernicus Marine Service products

_Author: Peter Kerins_  
_Created: 9 Jul 2021_  
_Environment: jupyterlab_  

Ocean Watch seeks to feature several biogeochemical parameters from the [Global Ocean Biogeochemistry Analysis and Forecast (GLOBAL_ANALYSIS_FORECAST_BIO_001_028) product](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=GLOBAL_ANALYSIS_FORECAST_BIO_001_028) of the [Copernicus Marine Service](https://marine.copernicus.eu/). The data will be mapped via Resource Watch. OW will also show time series of the parameters at various locations (representing outlets of rivers to the ocean).  

Presenting that time series data would typically require harvesting the information from the historical product rasters. However, the CMEMS [Pretty View](https://view-cmems.mercator-ocean.fr/GLOBAL_ANALYSIS_FORECAST_BIO_001_028) WebGIS tool displays just such time series (via the `Tools`). Furthermore, these are generated using the public-facing [WMS](https://www.ogc.org/standards/wms). The Copernicus Marine User Support Expert indicated [the relevant documentation](https://docs.geoserver.org/master/en/user/services/wms/reference.html#getfeatureinfo).

Constructing a properly formatted `GetFeatureInfo` request from scratch without examples was unsuccessful. However, spying on the Pretty View tool while making a time series request revealed [just such an example](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=chl&BBOX=53.26,14.26,53.2600001,14.2600001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z):

```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=chl
&BBOX=53.26,14.26,53.2600001,14.2600001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z
```

This query returns daily readings of `mass_concentration_of_chlorophyll_a_in_sea_water` at a depth of about half a meter for approximately the preceding year, at a location between Socotra and Yemen. 

A similar example closer to OW's needs can also be generated. [The query](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=o2&BBOX=31.01,31.74,31.010000100000003,31.7400001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z) for the full times series of monthly `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water` values at a half-meter of depth at a pixel near the mouth of the Nile:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

This template can be modified to accommodate OW needs.

# Service

Capabilities of service requested via:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetCapabilities
```

Key excerpt from response:
```xml
            <GetFeatureInfo>
                <Format>image/png</Format>
                <Format>text/xml</Format>
                <DCPType>
                    <HTTP>
                        <Get>
                            <OnlineResource xlink:type="simple" xlink:href="http://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly" />
                        </Get>
                    </HTTP>
                </DCPType>
            </GetFeatureInfo>
```
Data can only be returned in `text/xml` format, not the preferred JSON (`application/json`).

# Parameters

## Variable
- _mole_concentration_of_dissolved_molecular_oxygen_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=o2`  
- _mole_concentration_of_phosphate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=no3`  
- _mole_concentration_of_nitrate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=po4`  
  
Note: providing a comma-separated list of valid layers (eg `QUERY_lAYERS=o2,no3`) returns an exception (`400` response)


## Depth
- `elevation=-0.49402499198913574`
- `elevation=-1.5413750410079956`
- `elevation=-2.6456689834594727`
- `elevation=-3.8194949626922607`
- `elevation=-5.078224182128906`

## Bounding Box
`xmin`,`ymin` are just the point of selection; `xmax`, `ymax` are just minimum value plus 0.0000001

# Example Request

Oxygen
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=info_format=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

 # Workflow
 
 - Cycle through preexisting list of coordinates representing river mouths. For each coordinate pair:
 - 

In [ ]:
# preexisting:
# set of points representing river mouths
# table for data

# cycle
# for each point:
# for each chemical & depth:
# construct query
# pull data from wms
# hold xml response
# convert to something, probably dataframe
# (may choose to parallelize this process)
# append to larger existing dataframe

# at end: 
# clean existing persistent table
# dump new dataframe into table